# Atividade 04: Implementando uma Rede Convolucional por meio do TensorFlow

Nesta atividade, você irá treinar uma rede convolucional para realizar classificação de imagens utilizando o TensorFlow, e irá testá-la utilizando o dataset CIFAR-10.

Nesta atividade, o TensorFlow será examinado em **três diferentes níveis de abstração**, o que irá ajudá-lo a compreender melhor seu funcionamento e prepará-lo para utilizá-lo em outros projetos.

Nesta atividade, você irá:

- **preparar** o dataset CIFAR-10 para utilizar no treinamento com TensorFlow
- **trabalhar** diretamente sobre grafos de computação do TensorFlow (**Nível de Abstração 1 - Fundamentos do TensorFlow**)
- **utilizar** a API `tf.keras.Model` para se definir arquiteturas de redes neurais arbitrárias (**Nível de Abstração 2 - API Keras Model**)
- **utilizar** a API `tf.keras.Sequential` para se definir arquiteturas de redes *feed-forward* lineares de forma conveniente e **explorar** biblioteca de funções para construção de modelos mais flexíveis  (**Nível de Abstração 3 - API Keras Sequential + API Funcional**)

Detalhes sobre `Keras` serão apresentados mais adiante nesse *notebook*.

A tabela abaixo compara as abordagem mencionadas acima em relação ao grau de flexibilidade e de conveniência na implementação de grafos de computação:

| API           | Flexibilidade | Conveniência |
|---------------|-------------|-------------|
| Fundamentos      | Alta        | Baixa         |
| `tf.keras.Model`     | Alta        | Média      |
| `tf.keras.Sequential` | Baixa         | Alta        |

## TensorFlow
TensorFlow é um sistema para execução de grafos de computação sobre *tensores*, com um suporte nativo para realização de propagação retrógrada (*backpropagation*). Nele, trabalha-se com *tensores* que representam vetores/matrizes n-dimensionais análogas aos `ndarray` encontrados em `numpy`.

### Por que utilizar TensorFlow?

* Seu código irá executar em GPUs! Assim o treinamento pode ser realizado de forma mais rápida. A escrita de código próprio para execução em GPUs pode ser muito trabalhosa e está além do escopo dessa disciplina (porém, o uso de TensorFlow viabiliza a utilização de GPUs de forma transparente).
* Utilizar com um `framework` como o TensorFlow permite que você produza código de forma mais eficiente e eficaz ao invés de implementar tudo no zero. 
* TensorFlow representa um dos melhores `frameworks` para implementação de redes profundas 
* Você ganhará familiaridade com o tipo de código utilizado em aprendizagem profunda tanto na academia como na indústria. 

## Como você pode aprender e aprofundar em TensorFlow?

TensorFlow possui muitos tutoriais disponíveis, incluindo aqueles do próprio pessoal da [Google](https://www.tensorflow.org/get_started/get_started).

De toda forma, esse *notebook* irá guiá-lo sobre as etapas que você precisa realizar para treinar modelos no TensorFlow. No final desse *notebook*, você encontrará links para tutoriais auxiliares caso você deseje aprender mais ou necessite de esclarecimentos adicionais sobre tópicos que não sejam tratados de forma completa aqui.

**NOTA: Este *notebook* deve ser utilizado com a versão `2.2.0-rc3` do TensorFlow. A maioria dos exemplos na Web atualmente ainda utilizam versões `1.x`, portanto tenha cuidado para não se confundir quando consultar a documentação**.

## Instalando TensorFlow 2.0 (APENAS SE VOCÊ FOR TRABALHAR LOCALMENTE)

1. Tenha a última versão de Anaconda instalada em sua máquina.
2. Crie uma novo ambiente `conda` a partir do `Python 3.7`. Aqui, chamamos esse ambiente de `tf_20_env`.
3. Execute o comando: `source activate tf_20_env`
4. Em seguida, use `pip` para instalar TensorFlow 2.0 conforme descrito em: https://www.tensorflow.org/install

# Preparação

Primeiramente, vamos carregar os dados do dataset CIFAR-10. 

Nas atividades anteriores, você utilizou código específico para baixar e ler o dataset CIFAR-10; contudo o pacote `tf.keras.datasets` no TensorFlow fornece funções utilitárias para carga de vários datasets comuns.

Para o propósito dessa atividade, ainda será utilizado código para preprocessamento dos dados e iteração sobre eles em *minibatches*. O pacote `tf.data` no TensorFlow fornece ferramentas para automatizar esse processo, porém trabalhar com esse pacote está fora do escopo dessa atividade. Entretanto, o uso do pacote `tf.data` pode ser muito mais eficiente que a abordagem simples usada nesse *notebook* e você talvez deva considerar seu uso futuramente.

In [ ]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [ ]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

Você pode opcionalmente **usar GPU bastando setar o *flag* abaixo `USE_GPU` para `True`**.

## Usuários de Colab

Caso vocês esteja utilizando o `Colab`, você precisa ativar manualmente o uso de GPU. Você pode fazer isso selecionando `Ambiente de execução -> Alterar tipo de ambiente de execução` (em inglês, `Runtime -> Change runtime type`) e escolhendo `GPU` em `Acelerador de hardware` (em inglês, `Hardware Accelerator`). Observe que você deverá reexecutar as células desde o ínicio (caso faça essa mudança) pois o ambiente será reiniciado uma vez que faça essa alteração.

In [ ]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

# Fundamentos do TensorFlow
TensorFlow possui várias APIs de alto nível que o tornam uma ferramenta conveniente para se definir e treinar redes neurais. Algumas dessas construções serão cobertas mais adiante neste *notebook*. Nesta seção, vai se iniciar pela construção de um modelo com as primitivas mais básicas do TensorFlow de forma a ajudá-lo a compreender melhor o que se passa por detrás das APIs de mais alto nível.

**"Fundamentos do Tensorflow" são importantes para se compreender os blocos básicos do TensorFlow, porém muitos deles envolvem conceitos do TensorFlow 1.x.** Iremos trabalhar com módulos legados (*antigos*) como `tf.Variable`.

Dessa forma, leia com atenção e tente compreender as diferenças entre os módulos TensorFlow legados (antigos - 1.x) e os novos (2.0).

### Breve histórico sobre TensorFlow 1.x

TensorFlow 1.x é basicamente um `framework` para se trabalhar com **grafos de computação estáticos**. Nós em um grafo de computação representam **tensores** que irão armazenar vetores n-dimensionais quando o grafo estiver em execução; arestas no grafo representam funções que irão operar sobre os tensores quando o grafo for executado de forma a realizar alguma computação.

Antes do TensorFlow 2.0, era necessário se configurar o grafo em duas fases. Existem vários tutoriais que explicam esse processo de duas fases. O processo pode ser resumido da seguinte forma para o TensorFlow 1.x:
1. **Construir um grafo de computação que descreve a computação que se deseja realizar**. Esse estágio não realiza nenhuma computação; ele só cria uma representação simbólica da computação a ser realizada. Nesse estágio será(ão) geralmente definido(s) um ou mais `placeholder` que  representam as entradas do grafo de computação.
2. **Executar o gafo de computação várias vezes**. Cada vez que o grafo for executado (p.ex. para um passo do método de gradiente), deve-se especificar quais partes do grafo você deseja computar e fornecer um um dicionário `feed_dict` com valores concretos para cada entrada (`placeholder`) do grafo.

### Novo paradigma no TensorFlow 2.0
Agora, com o TensorFlow 2.0, pode-se simplesmente adotar uma forma funcional mais similar os demais códigos escritos em `Python` (e, também, mas similar em espiríto a um outro *framework* muito poderoso denominado `PyTorch`). Você pode obter mais detalhes em https://www.tensorflow.org/guide/eager.

A principal diferença entre as abordagens do TensorFlow 1.x e do TensorFlow 2.0 é que nesse último não se utiliza `tf.Session`, `tf.run`, `placeholder`, `feed_dict`. Para se obter informações adicionais sobre as diferenças entre as duas versões e como realizar a conversão entre elas, verifique o guia oficial de migração em https://www.tensorflow.org/alpha/guide/migration_guide

Mais adiante, o foco será sobre essa nova e mais simples abordagem.

### Função Flatten (achatamento)

Uma função de achatamento (`flatten`) pode ser definida de forma a realizar uma reformatação (*reshape*) dos dados de uma imagem para uso em uma rede completamente conectada.

No TensorFlow, dados para mapas de características convolucionais são geralmente armazenados em um tensor com dimensões N x H x W x C em que:

- N é o número de amostras de dados (tamanho do *minibatch*)
- H é a altura (*height*) do mapa de característica
- W é a largura (*width*) do mapa de característica
- C é o número de canais do mapa de característica

Essa é a forma correta de se representar dados quando se realiza algo como uma convolução 2D, que necessita de informação sobre a distribuição espacial das informações. Porém, quando se utiliza camadas afim completamente conectadas para se processar uma imagem, deseja-se que cada amostra (imagem) seja representada como um único vetor -- não é útil se separar os diferentes canais, linhas e colunas dos dados. Assim, utiliza-se a operação de achatamento (`flatten`) para colapsar os valores `H x W x C` em um único vetor.

Vale observar que a chamada para `tf.reshape` possui como alvo o formato `(N, -1)` significando que a primeira dimensão será mantida e a demais colapsadas em uma única segunda dimensão.

**NOTA**: TensorFlow e PyTorch diferem sobre o layout default de um tensor; TensorFlow usa N x H x W x C porém PyTorch usa N x C x H x W.

In [ ]:
def flatten(x):
    """    
    Input:
    - TensorFlow Tensor of shape (N, D1, ..., DM)
    
    Output:
    - TensorFlow Tensor of shape (N, D1 * ... * DM)
    """
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [ ]:
def test_flatten():
    # Construct concrete values of the input data x using numpy
    x_np = np.arange(24).reshape((2, 3, 4))
    print('x_np:\n', x_np, '\n')
    # Compute a concrete output value.
    x_flat_np = flatten(x_np)
    print('x_flat_np:\n', x_flat_np, '\n')

test_flatten()

### Fundamentos do TensorFlow: Definindo uma Rede de Duas Camadas
Agora, iremos implementar a primeira rede neural com TensorFlow: uma rede complementamente conectada com duas camadas usando ReLU e sem nenhum viés (termo independente) sobre o dataset CIFAR10. Por enquanto, utilizaremos apenas operações de baixo nível para definir a rede; mais adiante serão utilizadas abstrações mais elaboradas fornecidas pelo `tf.keras` para simplificar o processo.

O *forward pass* da rede será definido na função `two_layer_fc` que receberá tensores com as entradas e pesos da rede e retorna um tensor com os valores de *score*. 

Depois de definida a arquitetura de rede na função `two_layer_fc`, a implementação será testada checando-se as dimensões da saída.

**É muito importante que você leia e compreenda a implemtação abaixo.**

In [ ]:
def two_layer_fc(x, params):
    """
    A fully-connected neural network; the architecture is:
    fully-connected layer -> ReLU -> fully connected layer.
    Note that we only need to define the forward pass here; TensorFlow will take
    care of computing the gradients for us.
    
    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.

    Inputs:
    - x: A TensorFlow Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of TensorFlow Tensors giving weights for the
      network, where w1 has shape (D, H) and w2 has shape (H, C).
    
    Returns:
    - scores: A TensorFlow Tensor of shape (N, C) giving classification scores
      for the input data x.
    """
    w1, w2 = params                   # Unpack the parameters
    x = flatten(x)                    # Flatten the input; now x has shape (N, D)
    h = tf.nn.relu(tf.matmul(x, w1))  # Hidden layer: h has shape (N, H)
    scores = tf.matmul(h, w2)         # Compute scores of shape (N, C)
    return scores

In [ ]:
def two_layer_fc_test():
    hidden_layer_size = 42

    # Scoping our TF operations under a tf.device context manager 
    # lets us tell TensorFlow where we want these Tensors to be
    # multiplied and/or operated on, e.g. on a CPU or a GPU.
    with tf.device(device):        
        x = tf.zeros((64, 32, 32, 3))
        w1 = tf.zeros((32 * 32 * 3, hidden_layer_size))
        w2 = tf.zeros((hidden_layer_size, 10))

        # Call our two_layer_fc function for the forward pass of the network.
        scores = two_layer_fc(x, [w1, w2])

    print(scores.shape)

two_layer_fc_test()

### Fundamentos do TensorFlow: Definindo uma Rede Convolucional (ConvNet) de Três Camadas
Agora você deve completar a implementação da função `three_layer_convnet` que realiza o *forward pass* de uma rede convolucional de três camadas. A rede deve possuir a seguinte arquitetura:

1. Uma camada convolucional (com viés) com `channel_1` filtros, cada um deles com formato `KW1 x KH1`, e preenchimento (*zero-padding*) de tamanho 2
2. Não-linearidade ReLU
3. Uma camada convolucional (com viés) com `channel_2` filtros, cada um deles com formato `KW2 x KH2`, e preenchimento (*zero-padding*) de tamanho 1
4. Não-linearidade ReLU
5. Uma camada completamente conectada com viés, produzindo *scores* para `C` classes.

**DICA 1**: Para convoluções veja: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/nn/conv2d; tenha cuidado com o preenchimento (*padding*)!

**DICA 2**: Para viés veja: https://www.tensorflow.org/performance/xla/broadcasting

In [ ]:
def three_layer_convnet(x, params):
    """
    A three-layer convolutional network with the architecture described above.
    
    Inputs:
    - x: A TensorFlow Tensor of shape (N, H, W, 3) giving a minibatch of images
    - params: A list of TensorFlow Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: TensorFlow Tensor of shape (KH1, KW1, 3, channel_1) giving
        weights for the first convolutional layer.
      - conv_b1: TensorFlow Tensor of shape (channel_1,) giving biases for the
        first convolutional layer.
      - conv_w2: TensorFlow Tensor of shape (KH2, KW2, channel_1, channel_2)
        giving weights for the second convolutional layer
      - conv_b2: TensorFlow Tensor of shape (channel_2,) giving biases for the
        second convolutional layer.
      - fc_w: TensorFlow Tensor giving weights for the fully-connected layer.
        Can you figure out what the shape should be?
      - fc_b: TensorFlow Tensor giving biases for the fully-connected layer.
        Can you figure out what the shape should be?
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ############################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.            #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                              END OF YOUR CODE                            #
    ############################################################################
    return scores

Após definir o *forward pass* para a ConvNet de três camadas acima, execute o código da próxima célula para testar sua implementação. Semelhante a rede de duas camadas, o grafo será executado sobre tensores contendo zeros, portanto você deve assegurar que a função não tenham problemas de execução e que produz uma saída no formato correto.

Quando você executar essa função, `scores_np` deve ter dimensões `(64, 10)`.

In [ ]:
def three_layer_convnet_test():
    
    with tf.device(device):
        x = tf.zeros((64, 32, 32, 3))
        conv_w1 = tf.zeros((5, 5, 3, 6))
        conv_b1 = tf.zeros((6,))
        conv_w2 = tf.zeros((3, 3, 6, 9))
        conv_b2 = tf.zeros((9,))
        fc_w = tf.zeros((32 * 32 * 9, 10))
        fc_b = tf.zeros((10,))
        params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
        scores = three_layer_convnet(x, params)

    # Inputs to convolutional layers are 4-dimensional arrays with shape
    # [batch_size, height, width, channels]
    print('scores_np has shape: ', scores.shape)

three_layer_convnet_test()

### Fundamentos do TensorFlow: Passo de Treinamento

Agora definiremos a função `training_step` para realizar um único passo de treinamento. Isso exigirá 03 etapas básicas:

1. Computar a perda
2. Computar o gradiente da perda em relação a todos os pesos da rede
3. Fazer um passo de atulização de pesos utilizando SGD (*stochastic gradient descent*).

São necessárias algumas poucas funções do TensorFlow para se realizar isso:
- Para cálculo da perda por entropia cruzada, pode-se utilizar `tf.nn.sparse_softmax_cross_entropy_with_logits`: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits

- Para se obter a média da perda para as amostras de dados do *minibatch*, pode-se usar `tf.reduce_mean`:
https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/reduce_mean

- Para computar o gradiente da perda em relação aos pesos, pode-se usar `tf.GradientTape` (útil para execução rápida):  https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/GradientTape

- Já a modificação doa valores de pesos armazenados em um tensor, pode-se usar `tf.assign_sub` ("sub" é para subtração): https://www.tensorflow.org/api_docs/python/tf/assign_sub 


In [ ]:
def training_step(model_fn, x, y, params, learning_rate):
    with tf.GradientTape() as tape:
        scores = model_fn(x, params) # Forward pass of the model
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        total_loss = tf.reduce_mean(loss)
        grad_params = tape.gradient(total_loss, params)

        # Make a vanilla gradient descent step on all of the model parameters
        # Manually update the weights using assign_sub()
        for w, grad_w in zip(params, grad_params):
            w.assign_sub(learning_rate * grad_w)
                        
        return total_loss

In [ ]:
def train_part2(model_fn, init_fn, learning_rate):
    """
    Train a model on CIFAR-10.
    
    Inputs:
    - model_fn: A Python function that performs the forward pass of the model
      using TensorFlow; it should have the following signature:
      scores = model_fn(x, params) where x is a TensorFlow Tensor giving a
      minibatch of image data, params is a list of TensorFlow Tensors holding
      the model weights, and scores is a TensorFlow Tensor of shape (N, C)
      giving scores for all elements of x.
    - init_fn: A Python function that initializes the parameters of the model.
      It should have the signature params = init_fn() where params is a list
      of TensorFlow Tensors holding the (randomly initialized) weights of the
      model.
    - learning_rate: Python float giving the learning rate to use for SGD.
    """
    
    
    params = init_fn()  # Initialize the model parameters            
        
    for t, (x_np, y_np) in enumerate(train_dset):
        # Run the graph on a batch of training data.
        loss = training_step(model_fn, x_np, y_np, params, learning_rate)
        
        # Periodically print the loss and check accuracy on the val set.
        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss))
            check_accuracy(val_dset, x_np, model_fn, params)

In [ ]:
def check_accuracy(dset, x, model_fn, params):
    """
    Check accuracy on a classification model, e.g. for validation.
    
    Inputs:
    - dset: A Dataset object against which to check accuracy
    - x: A TensorFlow placeholder Tensor where input images should be fed
    - model_fn: the Model we will be calling to make predictions on x
    - params: parameters for the model_fn to work with
      
    Returns: Nothing, but prints the accuracy of the model
    """
    num_correct, num_samples = 0, 0
    for x_batch, y_batch in dset:
        scores_np = model_fn(x_batch, params).numpy()
        y_pred = scores_np.argmax(axis=1)
        num_samples += x_batch.shape[0]
        num_correct += (y_pred == y_batch).sum()
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### Fundamentos do TensorFlow: Inicialização

Pode-se utilizar a seguinte função para se inicializar as matrizes de pesos utilizando o método de normalização de Kaiming (He et al. 2015)

[1] He et al., *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [ ]:
def create_matrix_with_kaiming_normal(shape):
    if len(shape) == 2:
        fan_in, fan_out = shape[0], shape[1]
    elif len(shape) == 4:
        fan_in, fan_out = np.prod(shape[:3]), shape[3]
    return tf.keras.backend.random_normal(shape) * np.sqrt(2.0 / fan_in)

### Fundamentos do TensorFlow: Treinando uma Rede de Duas Camadas

Agora, estamos prontos para utilizar todas as funções definidas acima no treinamento de uma rede de duas camadas completamente conectada sobre o dataset CIFAR-10.

Basta então definir uma função para inicializar os pesos do modelo e chamar, em seguida, a função `train_part2`.

Para se definir os pesos da rede é necessário se introduzir uma outra peça importante da API do TensorFLow: `tf.Variable`. 

Uma variável do TensorFlow é um tensor cujo valor é armazenado em um grafo e que persiste entre várias execuções do grafo de computação; entretanto diferente de constantes definidas com `tf.zeros` ou `tf.random_normal`, os valores de uma variável podem mudar na medida que o grafo executa; essas alterações irão persistir entre diferentes execuções do gafo. Assim, parâmetros da rede que são aprendidos são geralmente armazenados em variáveis.

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 40% após uma época de treinamento.

In [ ]:
def two_layer_fc_init():
    """
    Initialize the weights of a two-layer network, for use with the
    two_layer_network function defined above. 
    You can use the `create_matrix_with_kaiming_normal` helper!
    
    Inputs: None
    
    Returns: A list of:
    - w1: TensorFlow tf.Variable giving the weights for the first layer
    - w2: TensorFlow tf.Variable giving the weights for the second layer
    """
    hidden_layer_size = 4000
    w1 = tf.Variable(create_matrix_with_kaiming_normal((3 * 32 * 32, 4000)))
    w2 = tf.Variable(create_matrix_with_kaiming_normal((4000, 10)))
    return [w1, w2]

learning_rate = 1e-2
train_part2(two_layer_fc, two_layer_fc_init, learning_rate)

### Fundamentos do TensorFlow: Treinando uma ConvNet de Três Camadas

Agora, você deve usar o TensorFlow para treinar uma ConvNet de 3 camadas sobre o dataset CIFAR-10.

Você deve implementar a função `three_layer_convnet_init`. Lembre-se de que a arquitetura da rede é a seguinte:

1. Camada convolucional (com viés) com 32 filtros 5x5, e com preenchimento (*zero-padding*) de tamanho 2
2. Não-linearidade ReLU
3. Camada convolucional (com viés) com 16 filtros 3x3, e com preenchimento (*zero-padding*) de tamanho 1
4. Não-linearidade ReLU
5. Camada completamente conectada (com viés) para computar *scores* para 10 classes

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 43% após uma época de treinamento.

In [ ]:
def three_layer_convnet_init():
    """
    Initialize the weights of a Three-Layer ConvNet, for use with the
    three_layer_convnet function defined above.
    You can use the `create_matrix_with_kaiming_normal` helper!
    
    Inputs: None
    
    Returns a list containing:
    - conv_w1: TensorFlow tf.Variable giving weights for the first conv layer
    - conv_b1: TensorFlow tf.Variable giving biases for the first conv layer
    - conv_w2: TensorFlow tf.Variable giving weights for the second conv layer
    - conv_b2: TensorFlow tf.Variable giving biases for the second conv layer
    - fc_w: TensorFlow tf.Variable giving weights for the fully-connected layer
    - fc_b: TensorFlow tf.Variable giving biases for the fully-connected layer
    """
    params = None
    ############################################################################
    # TODO: Initialize the parameters of the three-layer network.              #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return params

learning_rate = 3e-3
train_part2(three_layer_convnet, three_layer_convnet_init, learning_rate)

# API Keras Model

Implementar uma rede neural utilizando a API de baixo nível do TensorFlow é uma boa forma de entender como o TensorFlow funciona, porém é um pouco inconveniente, pois se deve manualmente manter e acompanhar todos os tensores armazenando parâmetros que serão aprendidos. Isso é simples para uma rede pequena, mas pode rapidamente se tornar impraticável para modelos grandes e complexos.

Felizmente, o TensorFlow 2.0 fornece APIs de alto nível como `tf.keras` que facilitam a construção de modelos a partir de camadas modulares. Além disso, o TensorFlow 2.0 utilizar a execução rápida que avalia operações de forma imediata sem a necessidade de construção explíta de nenhum grafo de computação. Isso torna mais fácil de se escrever e depurar modelos.

Nessa parte desse *notebook*, você irá definir um modelo de rede neural usando a API `tf.keras.Model`. Para implementar seu próprio modelo, você precisará do seguinte:

1. Definir uma nova classe que seja herdeira de `tf.keras.Model`. Você deve dar um nome intuitivo a sua classe que sirva para descrevê-la, como `TwoLayerFC` ou `ThreeLayerConvNet`.
2. No método de inicialização `__init__()` para sua nova classe, você deve definir todas as camadas como atributos de classe. O pacote `tf.keras.layers` fornece muitas camadas comuns as redes neurais, como `tf.keras.layers.Dense` para camadas completamente conectadas e `tf.keras.layers.Conv2D` para camadas convolucionais. Internamente, essas camadas irão construir variáveis (tensores) para quaisquer parâmetros que devam ser aprendidos. **AVISO: Não se esqueça de chamar `super(YourModelName, self).__init__()` como a primeira instrução de seu inicializador!**
3. Implementar o método `call()` para sua classe, que será responsável pelo cálculo do *forward pass* de seu modelo, e estabelece a *conectividade* entre os elementos de sua rede. Camadas definidas em no método `__init__()` são utilizadas para implementar o método `__call__()`. Assim, as camadas podem seu usadas para transformar tensores de entrada em tensores de saída. Não define nenhuma nova camada no método `call()`; qualquer camada que você deseje utilizar no *forward pass* deve ser definida no método `__init__()`.

Após definir sua subclasse de `tf.keras.Model`, você pode criar uma instância dela utilizá-la como um modelo.

### Exemplo de Subclasse de Keras Model: Rede de Duas Camadas

Eis agora um exemplo concreto de uso da API `tf.keras.Model` para se definir uma rede de duas camadas. Existem alguns pequenos pontos da API a se prestar atenção aqui:

Será utilizado um objeto `Initializer` para definir valores iniciais para os parâmetros/pesos (a serem aprendidos) nas camadas; em particular `tf.initializers.VarianceScaling` fornece um comportamento similar ao método de inicialização de Kaiming utilizado anteriormente nesse *notebook* (ver He et al. 2015). Você pode encontrar mais detalhes em https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/initializers/VarianceScaling

Serão utilizado objetos `tf.keras.layers.Dense`para representar as duas camadas completamente conectadas do modelo. Além de multiplicar suas entradas por uma matriz de pesos e adicionar um vetor de viés, essas camadas também podem aplicar uma não-linearidade para você. Para a primeira camada, especifica-se um função de ativação ReLU passando o parâmetro `activation='relu'` para o construtor; já a segunda camada usa uma função de ativação *softmax*. Por fim, utiliza-se `tf.keras.layers.Flatten` para achatar o tensor de entrada antes das camadas completamente conectadas.

In [ ]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

### Uso da API Keras Model: Definindo uma ConvNet de Três Camadas

Agora, você deve implementar uma ConvNet de 3 camadas usando a API `tf.keras.Model`. Seu modelo deve a mesma arquitetura utilizada anteriormente neste *notebook*:

1. Camada convolucional com filtros 5x5 e com preenchimento (*zero-padding*) de tamanho 2
2. Não-linearidade ReLU
3. Camada convolucional com filtros 3x3 e com preenchimento (*zero-padding*) de tamanho 1
4. Não-linearidade ReLU
5. Camada completamente conectada (com viés) para computar *scores* para classes
6. Não-linearidade Softmax

Você deve inicializar os pesos de sua rede utilizando o mesmo método usado na rede de duas camada acima.

**DICA**: Para conseguir implementar, explore a documentação de `tf.keras.layers.Conv2D` e `tf.keras.layers.Dense`:

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [ ]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

Uma vez que você termine a implementaçaõ da classe `ThreeLayerConvNet` acima, você pode executar o seguinte código para assegurar que sua implementação não contém de execução e que produz saída no formato esperado.

In [ ]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

### Uso da API Keras Model: Treinamento Rápido

Apesar de moelos em Keras possuirem um método de treinamento embutido (por meio de `model.fit`), as vezes é necessário se *customizar* o treinamento. A seguir, vocÊ verá um exemplo de um método de treinamento implementado por meio de execução rápida.

Em particular, observe o uso de `tf.GradientTape`. Diferenciação automática é utilizada para implementar a propagação retrógrada em `frameworks` como o TensorFlow. Durante a execução rápida, `tf.GradientTape` é usado para cálculo de gadientes. 

Além disso, TensorFlow 2.0 possui um mecanismo simplificadopara avaliação de métricas por meio do módulo `tf.keras.metrics`. Cada métrica é um objeto. Pode-se adicionar observações por meio de `update_state()` e eliminar todas elas usando `reset_state()`. Para se obter o valor corrente de uma métrica, usa-se o método `result()` sobre o respectivo objeto. 

In [ ]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

### Uso da API Keras Model: Treinando uma Rede de Duas Camadas

Agora, pode-se utilizar as ferramentar definidas anteriormente para se treinar uma rede de duas camadas sobre o dataset CIFAR-10. 

Definiu-se os métodos `model_init_fn` e `optimizer_init_fn` que controem o modelo e executam a otimização do mesmo, respectivamente. 

Deseja-se treinar o modelo por meio do SGD (*stochastic gradient descent*) sem uso de momento, portanto, utiliza-se a função `tf.keras.optimizers.SGD`; você pode [ler mais a esse respeito aqui](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD).

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 40% após uma época de treinamento.

In [ ]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

### Uso da API Keras Model: Treinando uma ConvNet de Três Camadas

Agora você deve utilizar as ferramentas definidas anteriormente para treinar uma ConvNet de 3 camadas sobre o dataset CIFAR-10. Sua ConvNet deve possuir 32 filtros na primeira camada convolucional e 16 filtros na segunda camada.

Para treinar o modelo, você deve adotar o método do gradiente com o momento de Nesterov de 0,9.  

**DICA**: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 50% após uma época de treinamento.

In [ ]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

# API Keras Sequential

As implementações com a API `tf.keras.Model` permitem que você defina  modelos com qualquer número de camadas e com uma conectividade arbitrária entre elas.

Contudo, para muitos modelos você não necessita de tamanha flexibilidade - grande parte dos modelos pode ser expressa como uma pilha sequencial de camadas em que a saída de uma camada é usada como entrada da próxima.
Caso seu modelo se encaixe nesse padrão, então existe um forma ainda mais fácil de se definir o modelo por meio do uso da API `tf.keras.Sequential`. Você não precisa escrever nenhuma classe customizada; basta chamar o construtor `tf.keras.Sequential` com uma lista contendo a senquência de camadas.

Uma dificuldade no uso de `tf.keras.Sequential` é que você deve definir as dimensões da entrada do modelo e informar isso (por meio do valor de `input_shape`) à primeira camada de seu modelo.

### Uso da API Keras Sequential: Rede de Duas Camadas

Agora, a rede completamente conectada de duas camadas será reescrita usando `tf.keras.Sequential`, e treinada com o método definido acima.

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 40% após uma época de treinamento.

In [ ]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

### Abstraindo o Método de Treinamento

Nos exemplos anteriores, foi utilizado um método de treinamento customizado para se treinar os modelos (p.ex. `train_part34`). Excrever seu próprio método de treinamento só é necessário se você desejar mais flexibilidade e controle durante o treinamento de seu modelo. Alternativamente, você pode utilizar de métodos prontos da API como `tf.keras.Model.fit()` e `tf.keras.Model.evaluate` para treinar e avaliar um modelo. Nesse caso, você deve lembrar de configurar antes seu modelo para treinamento chamando `tf.keras.Model.compile`.

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 42% após uma época de treinamento.

In [ ]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

### Uso da API Keras Sequential: ConvNet de Três Camadas

Agora você deve usar `tf.keras.Sequential` para reimplementar a  arquitetura da ConvNet de 3 camadas anterior. Como lembrete, seu modelo deve possuir a seguinte arquitetura:

1. Camada convolucional com filtros 5x5 e com preenchimento (*zero-padding*) de tamanho 2
2. Não-linearidade ReLU
3. Camada convolucional com filtros 3x3 e com preenchimento (*zero-padding*) de tamanho 1
4. Não-linearidade ReLU
5. Camada completamente conectada (com viés) para computar *scores* para classes
6. Não-linearidade Softmax

Você deve inicializar os pesos do modelo usando `tf.initializers.VarianceScaling` como antes e o modelo deve ser treinado com 
momento de Nesterov de 0,9.

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 45% após uma época de treinamento.

In [ ]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Você também pode treinar esse modelo utilizando o método pronto da API do TensorFlow.

In [ ]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

##  API Funcional
### Demonstração com uma Rede de Duas Camadas 

Nas seções anteriores desse *notebook*, vimos como usar `tf.keras.Sequential` para empilhar camadas de forma a se construir rapidamente modelos simples. Porém isto possui um custo: a perda de flexibilidade.

Em alguns casos, deseja-se escrever modelos complexos que possuem um fluxo de dados não-sequencial: uma camada pode ter **múltiplas entradas e/ou saídas**, como, por exemplo, concatenar as saídas de duas camadas anteriores para se fornecer como entrada para uma próxima! Alguns exemplos disso são conexões residuais e os blocos densos da ResNet e da DenseNet, respectivamente.

Nesses casos, pode-se utilizar a API funcional do Keras para se escrever modelos com topologias complexas como:

 1. Modelos com múltiplas entradas
 2. Modelos com múltiplas saídas
 3. Modelos com camadas compartilhadas (a mesma camada chamada várias vezes)
 4. Modelos com fluxos de dados não-sequenciais (p.ex. conexões residuais)

Para a escrita de um modelo com a API funcional, você deve criar uma instância de `tf.keras.Model` e explicitamente fornecer os tensores de entrada e de saída para seu modelo. 

In [ ]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

### Uso da API Keras Funcional: Treinando uma de Duas Camadas

Agora você pode treinar a rede de duas camadas construída (ver acima) utilizando a API funcional.

Você não precisa ajustar nenhum hiperparâmetro, mas (mesmo assim) deve alcançar uma acurácia de validação acima de 40% após uma época de treinamento.

In [ ]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

# Desafio Final: Treinar uma ConvNet sobre o Dataset CIFAR-10

Nesta seção você poderá experimentar com uma arquitetura de ConvNet que desejar treinar sobre o dataset CIFAR-10.

Você deve experimentar diferentes arquiteturas, hoperparâmetros, funções de perda, regularização ou quaisquer outros recursos que deseje para treinar um modelo que alcance **pelo menos 70%** de acurácia sobre o conjunto de **validação** dentro de 10 épocas.

Você pode utilizar as funções de treinamento prontas da API, a função `train_part34` dada acima ou mesmo implementar sua própria função.

Ao final, é importante que você consiga descrever o que você fez para alcançar seu resultado!

### Algumas ideias que você pode experimentar:
- **Tamanho de filtros**: Acima foram usados filtros 5x5 e 3x3; esses tamanhos são ideiais?
- **Número de filtros**: Acima foram usados 16 e 32 filtros. Será que um aumento ou redução dessas quantidades produziria um resultado melhor?
- **Agrupamento (*Pooling*)**: Acima não se utilizou de nenhuma camada de agrupamento (*pooling*). Isso poderia melhorar o modelo?
- **Normalização**: Será que o modelo seria melhor se fosse adotada alguma técnica de normalização, como, por exemplo, *batch normalization*?
- **Arquitetura da Rede**: A ConvNet anterior possui apenas 3 camadas. Um modelo mais profundo teria resultados melhores?
- **Agrupamento Médio Global (*Global average pooling*)**: Se, ao invés de achatamento (*flattening*) depois da última camada convolucional, fosse utilizado um agrupamento médio global (*global average pooling*) o resultado seria melhor? Essa estratágia é usada por exemplo na GoogLeNet e na ResNet.
- **Regularização**: Será que o uso de alguma forma de regularização (como, decaimento de taxa e *dropout*, por exemplo) melhoria o resultado do modelo?

### NOTA: *Batch Normalization / Dropout*
Se você for utilizar *batch normalization* e *drpout*, lembre-se de fornecer o parâmetro `is_training=True` caso utilize o método de treinamento  `train_part34()`. Camadas **BatchNorm** e **Dropout** possuem comportamentos diferentes durante o treinamento e seu posterior uso na inferência.
`training` é um argumento específico reservado para esse propósito em qualquer função `call()` da API  `tf.keras.Model`. Mais informações sobre isso em https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods
https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

### Dicas para Treinamento
Para cada arquitetura de rede que você experimentar, você deve ajustar a taxa de aprendizado e demais hiperparâmetros. Quando realizar isso, algumas aspectos devem ser considerados:

- Se os hiperparâmetros são bons, você deve observar melhorias dentro de algumas centenas de iterações
- Lembre-se de usar uma abordagem de refinamento sucessivos (*coarse-to-fine*) nos ajustes de hiperparâmetros: inicie com intervalos grandes e faça poucas iterações de forma a determinar combinações promissoras para serem exploradas mais cuidadosamente a seguir 
- Uma vez que tenha encontrado alguns conjuntos de parâmetros que parecem funcionar, faça uma busca mais refinada em torno desses valores de parâmetros. Talvez seja necessário treinar por um número maior de épocas
- Você deve usar o conjunto de validação para ajuste de hiperparâmetros (**nunca o conjunto de teste!**), e deixar reservado o conjunto de teste para avaliar sua arquitetura final obtida com os melhores parâmetros selecionados por meio do conjunto de validação

### Indo adiante e além...
Caso você deseje existem inúmeros recursos e estratégias adicionais que você pode experimentar para melhorar a performance de seu modelo. Você **não precisa** implementar nenhuma delas, mas não deixe a diversão de lado, caso tenha tempo disponível!

- Otimizadores alternativos: você pode experimentar usar Adam, Adagrad, RMSprop, etc.
- Funções de ativação alternativas como *Leaky ReLU*, *Parametric ReLU*, ELU, ou MaxOut.
- Construir conjuntos de modelos (*model ensembles*)
- Realizar *data augmentation*
- Explorar novas arquiteturas
  - [ResNets](https://arxiv.org/abs/1512.03385) em que a entrada da camada anterior é adicionada a saída
  - [DenseNets](https://arxiv.org/abs/1608.06993) em que entradas de camadas anteriores são concatenadas
  - [Eis um *blog* interessante com mais ideias](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)
  
### Tenha uma boa (e divertida) experiência de treinamento! 

In [ ]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)